In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

df0 = pd.read_csv(r"E:\NE_glb_cities_era5_ae_1023.csv")
df0.head()

,Unnamed: 0.1,Unnamed: 0,latitude,longitude,utc_time,xco2,baseXCO2,NE,local_time,year,...,surface_pressure,leaf_area_index_high_vegetation,leaf_area_index_low_vegetation,volumetric_soil_water_layer_1surface_net_solar_radiation_hourly,surface_net_thermal_radiation_hourly,volumetric_soil_water_layer_1,surface_net_solar_radiation_hourly,uv10m,LAI,emission
0,0,999250,34.164528,-118.48089,2023-02-06 18:00:11.725021952,421.83026,423.435196,-1.604935,2023-02-06 10:00:11.725021952-08:00,2023,...,98499.660156,4.792358,1.024170,NaN,-367060.0,0.282516,1462010.0,2.731091,5.816528,42066.082
1,1,999251,34.170770,-118.49996,2023-02-06 18:00:11.752018432,421.61475,423.435949,-1.821203,2023-02-06 10:00:11.752018432-08:00,2023,...,98499.660156,4.792358,1.024170,NaN,-367060.0,0.282516,1462010.0,2.731091,5.816528,42066.082
2,2,999252,34.140636,-118.34925,2023-02-06 18:00:11.896012800,421.81088,423.432313,-1.621430,2023-02-06 10:00:11.896012800-08:00,2023,...,100632.660156,4.867798,1.668091,NaN,-379043.0,0.270538,1491254.0,2.343180,6.535889,80558.660
3,3,999253,34.147100,-118.36807,2023-02-06 18:00:11.923009280,421.96448,423.433093,-1.468615,2023-02-06 10:00:11.923009280-08:00,2023,...,100716.660156,4.254028,0.794189,NaN,-376133.0,0.275803,1509558.0,2.847821,5.048218,80558.660
4,4,999254,34.153610,-118.38710,2023-02-06 18:00:11.950005504,422.16742,423.433878,-1.266459,2023-02-06 10:00:11.950005504-08:00,2023,...,99137.660156,5.000488,0.993042,NaN,-374100.0,0.267105,1470882.0,2.783441,5.993530,80558.660


In [4]:
df=df0

In [28]:
df.loc[(df['LAI'].isna()) & (df['emission'] == 0), 'emission'] = np.nan
df.loc[(df['LAI'].isna()) & (df['surface_net_solar_radiation_hourly'] == 0), 'surface_net_solar_radiation_hourly'] = np.nan

In [23]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'latitude', 'longitude', 'utc_time',
       'xco2', 'baseXCO2', 'NE', 'local_time', 'year', 'month', 'hour',
       'geometry', 'index_right', 'COUNTRY', 'NAME_1', 'NAME_2',
       'local_time_str', 'u_component_of_wind_10m', 'v_component_of_wind_10m',
       'dewpoint_temperature_2m', 'temperature_2m', 'soil_temperature_level_1',
       'surface_pressure', 'leaf_area_index_high_vegetation',
       'leaf_area_index_low_vegetation',
       'volumetric_soil_water_layer_1surface_net_solar_radiation_hourly',
       'surface_net_thermal_radiation_hourly', 'volumetric_soil_water_layer_1',
       'surface_net_solar_radiation_hourly', 'uv10m', 'LAI', 'emission'],
      dtype='object')

In [5]:
# 国家到大洲的字典
continents = {
    "Europe": [
        'France', 'Croatia', 'Bosnia and Herzegovina', 'Albania', 'Spain',
        'Poland', 'Czechia', 'Slovakia', 'Ukraine', 'Italy', 'Greece',
        'Portugal', 'Germany', 'Switzerland', 'United Kingdom', 'Romania',
        'Serbia', 'Hungary', 'Slovenia', 'Austria', 'Bulgaria'
    ],
    "Asia": [
        'Turkey', 'Jordan', 'Russia', 'China', 'Tajikistan', 'Iran',
        'Saudi Arabia', 'South Korea', 'Lebanon', 'Syria', 'Kyrgyzstan',
        'Turkmenistan', 'Iraq', 'Azerbaijan', 'Japan', 'Uzbekistan',
        'Kazakhstan', 'Malaysia', 'Georgia', 'Philippines', 'Thailand',
        'Myanmar', 'Afghanistan', 'Oman', 'Vietnam', 'Mongolia', 'Pakistan',
        'United Arab Emirates', 'Bangladesh', 'Indonesia',  'Yemen',
        'Laos', 'North Korea', 'Cambodia'
    ],
    "North America": [
        'USA', 'México', 'Canada', 'Honduras', 'Cuba', 'Haiti',
        'Guatemala', 'Costa Rica', 'Dominican Republic', 'Nicaragua', 'Panama',
        'El Salvador'
    ],
    "South America": [
        'Venezuela', 'Ecuador', 'Colombia', 'Brazil', 'Argentina', 'Peru',
        'Bolivia', 'Chile', 'Uruguay', 'Paraguay', 'Suriname', 'French Guiana'
    ],
    "Africa": [
        'Tunisia', 'Algeria', 'Nigeria', 'Egypt', 'Morocco', 'Sudan',
        'Ethiopia', 'Senegal', 'Burkina Faso', 'Ghana', 'Benin', 'Mali',
        'Djibouti', 'Eritrea', 'Somalia', 'Uganda', 'Cameroon', 'Niger', 'Chad',
        'Mauritania', 'Gabon', 'Guinea-Bissau', 'Sierra Leone', 'Guinea',
        'Central African Republic', 'South Africa', 'Angola',
        'Democratic Republic of the Congo', 'Republic of the Congo',
        'Zambia', 'Namibia', 'Zimbabwe', 'Botswana', 'Kenya', 
        'Mozambique', "Côte d'Ivoire",  'Malawi', 'Madagascar', 'Rwanda',
        'Tanzania', 'Togo'
    ],
    "Oceania": [
        'Australia', 'New Zealand', 'Papua New Guinea'
    ]
}
# 将国家到大洲的字典转换为一个国家到大洲的反向字典
continent_map = {}
for continent, countries in continents.items():
    for country in countries:
        continent_map[country] = continent
# 创建新的列 'continent'，并使用 map 函数将国家映射到对应的大洲
df['continent'] = df['COUNTRY'].map(continent_map)


In [6]:
# 按照1s分组
df['datetime'] = pd.to_datetime(df['utc_time']).astype(str).str[:19]
df['datetime'] = pd.to_datetime(df['datetime'])

# 按照 name 和分组合并
df = df.groupby(['datetime']).agg({
    'longitude': 'mean',
    'latitude': 'mean',
    'NE': 'mean',                    
    'emission': 'mean', 
    'uv10m': 'mean',
    'temperature_2m': 'mean', 
    'LAI':'mean',
    'volumetric_soil_water_layer_1':'mean', 
    'surface_net_solar_radiation_hourly':'mean',
     'surface_pressure':'mean',
    'dewpoint_temperature_2m':'mean',
    'COUNTRY': lambda x: x.mode()[0] if not x.mode().empty else None, 
    'NAME_1': lambda x: x.mode()[0] if not x.mode().empty else None, 
    'NAME_2': lambda x: x.mode()[0] if not x.mode().empty else None,
    'continent': lambda x: x.mode()[0] if not x.mode().empty else None
}).reset_index(drop=False)

len(df)

237621

In [7]:
# 创建新的 'lat' 和 'lon' 列，四舍五入 latitude 和 longitude 到小数点后一位
df['lat'] = df['latitude'].round(1)
df['lon'] = df['longitude'].round(1)

# 查看结果
print(df[['latitude', 'longitude', 'lat', 'lon']].head())

    latitude   longitude   lat    lon
0  22.949263  116.166620  22.9  116.2
1  22.953365  116.191460  23.0  116.2
2  23.484223  116.693225  23.5  116.7
3  23.496234  116.725445  23.5  116.7
4  24.362333  117.914950  24.4  117.9


In [8]:
df['date'] = pd.to_datetime(df['datetime']).dt.date

# 查看结果
print(df[['datetime', 'date']].head())

             datetime        date
0 2019-08-06 04:06:41  2019-08-06
1 2019-08-06 04:06:42  2019-08-06
2 2019-08-06 04:06:52  2019-08-06
3 2019-08-06 04:06:53  2019-08-06
4 2019-08-06 04:07:16  2019-08-06


In [4]:
'''
df0['LAI'] = df0['leaf_area_index_high_vegetation'] + df0['leaf_area_index_low_vegetation']
df0 = df0.dropna(subset=['longitude','latitude',
    'dewpoint_temperature_2m', 'temperature_2m', 'surface_pressure', 
    'LAI', 'volumetric_soil_water_layer_1', 'emission', 
    'surface_net_solar_radiation_hourly', 'uv10m'
])

# 按照 era5分组合并
df0 = df0.groupby(['uv10m','temperature_2m', 'LAI', 'volumetric_soil_water_layer_1', 'surface_net_solar_radiation_hourly']).agg({
    'longitude': 'mean',
    'latitude': 'mean',
    'NE': 'mean',                    
    'emission': 'mean',
    'month':'mean',
    'date':'first'
}).reset_index(drop=False)

len(df0)'''

230483

    latitude   longitude   lat    lon
0  45.629102   25.670054  45.6   25.7
1  37.880790   28.148710  37.9   28.1
2  20.667171 -103.236002  20.7 -103.2
3  40.596012   -3.528849  40.6   -3.5
4 -29.904759  -51.181551 -29.9  -51.2


In [ ]:
import pandas as pd
import numpy as np
import os
import xarray as xr


# 初始化辐射通量列
df['Solar_Radiation_Flux'] = np.nan

# 遍历存储辐射数据的文件夹
radiation_folder = r"E:\radiation_ecmwf"
for subdir, dirs, files in os.walk(radiation_folder):
    for file in files:
        # 只处理.nc文件
        if file.endswith('.nc'):
            # 打开NetCDF文件
            file_path = os.path.join(subdir, file)
            print(f"Opening file: {file_path}")
            with xr.open_dataset(file_path) as ds:
                # 获取时间变量的值
                time_value = ds['time'].values[0].astype(str)  # 获取第一个时间点并转换为字符串
                date_value = pd.to_datetime(time_value).date()  # 转换为日期
                print(f"Time value in file: {date_value}")

                # 遍历df0中的每一行
                for idx, row in df.iterrows():
                    lat = row['lat']
                    lon = row['lon']
                    date = row['date']

                    # 直接比较日期字符串
                    if date == date_value:
                        # 查找最近的经纬度
                        try:
                            nearest_lat_idx = (np.abs(ds['lat'].values - lat)).argmin()
                            nearest_lon_idx = (np.abs(ds['lon'].values - lon)).argmin()

                            # 获取对应的Solar_Radiation_Flux值
                            radiation_value = ds['Solar_Radiation_Flux'].isel(lat=nearest_lat_idx, lon=nearest_lon_idx, time=0).values
                            df.at[idx, 'Solar_Radiation_Flux'] = radiation_value

                        except Exception as e:
                            print(f"Error for lat: {lat}, lon: {lon}, date: {date} - {e}")

# 保存结果到新的CSV文件
df.to_csv(r"E:\NE_glb_cities_era5_ae_rad_1103_agg.csv", index=False)


Opening file: E:\radiation_ecmwf\71bdd5326104d6676a47be1c37296314\Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20230101_final-v1.1.nc
Time value in file: 2023-01-01
Opening file: E:\radiation_ecmwf\71bdd5326104d6676a47be1c37296314\Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20230102_final-v1.1.nc
Time value in file: 2023-01-02
Opening file: E:\radiation_ecmwf\71bdd5326104d6676a47be1c37296314\Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20230103_final-v1.1.nc
Time value in file: 2023-01-03
Opening file: E:\radiation_ecmwf\71bdd5326104d6676a47be1c37296314\Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20230104_final-v1.1.nc
Time value in file: 2023-01-04
Opening file: E:\radiation_ecmwf\71bdd5326104d6676a47be1c37296314\Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20230105_final-v1.1.nc
Time value in file: 2023-01-05
Opening file: E:\radiation_ecmwf\71bdd5326104d6676a47be1c37296314\Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20230106_final-v1.1.nc
Time value in file: 2023-01-06
Opening file: E:\radia

Time variable:
<xarray.DataArray 'time' (time: 1)>
array(['2023-01-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2023-01-01
Attributes:
    standard_name:  time
    long_name:      time
    axis:           T
Time variable type: datetime64[ns]
Time values: ['2023-01-01T00:00:00.000000000']


In [13]:
dd = df0[(df0['year']==2023) & (df0['month']==1)]
dd['Solar_Radiation_Flux'].max() #['date'].unique()

31559576.0

In [ ]:
data_list